# 1. pytorch 2.x으로 설치하기


실습하기에 앞서 파이토치 2.x으로 설치 필요!   
Colab에서는 바로 사용할 수 있다.

https://pytorch.org/examples/

In [ ]:
import torch

torch.__version__

파이토치는 cpu, gpu 모두 사용할 수 있으며,  
colab의 경우 기본적으로 gpu가 설치된 버전으로 사용이 가능  
torch 버전을 출력해보면 나오는 버전+환경을 통해 확인이 가능하다.

In [ ]:
# pytorch에서 사용할 함수들 호출하기
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

먼저, datasets안에 들어있는 몇 가지 예제 데이터들 중에,  
fashion_mnist 데이터를 사용해보겠습니다.

In [ ]:
train_dataset = datasets.MNIST('./data', train=True, download=True)

pytorch 데이터셋의 경우 PIL.Image와 라벨로 이루어져 있다

In [ ]:
len(train_dataset)

In [ ]:
train_dataset[0]

In [ ]:
train_dataset[0][0]

In [ ]:
unique_class = np.unique([label for img, label in train_dataset])
print(unique_class)
len(unique_class)

지금은 이미지로 보이지만, 컴퓨터를 학습시키기 위해선 숫자로 바꿔주는 과정이 필요하다.

In [ ]:
transform = transforms.Compose([
        transforms.ToTensor(), # 이미지를 tensor 자료형으로 변경
        transforms.Normalize((0.1307,), (0.3081,)) # 정규화
        ])

train_dataset.transform = transform

기본적으로 이미지를 tensor 자료형으로 변경할 경우  
한 이미지는 이미지 채널 수 (Gray or RGB), width, height 순의 배열로 나타난다.

In [ ]:
train_dataset[0][0].shape

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.imshow(train_dataset[0][0].reshape(28, 28, 1)) # width, height, color channels
plt.colorbar()
plt.show()

pytorch의 데이터를 학습시키기 위해선 DataLoader로 바꿔주는 과정이 필요하다.  
참고로 dataset의 경우 데이터, 라벨로 이루어져 있어야 하고 iterable해야한다.

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)

# 숫자 이미지 예측하기

In [ ]:
plt.figure(figsize=(20,20))
for i in range(100):
    plt.subplot(10,10,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(train_dataset[i][0].reshape(28, 28, 1), cmap=plt.cm.binary)
    plt.xlabel(train_dataset[i][1])
plt.show()

테스트 데이터 정의

In [ ]:
test_dataset = datasets.MNIST('./data', train=False, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32)

### 모델 구축

In [ ]:
# 기본 모델 = 완전연결계층 = Linear
class MyFirstModel(nn.Module):
    def __init__(self):
        super(MyFirstModel, self).__init__()

        self.model = nn.Sequential(
            nn.Flatten(),
            nn.Linear(784, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 10),
            nn.Softmax()
        )

    def forward(self, x):
        output = self.model(x)
        return output

In [ ]:
model = MyFirstModel()

### 모델 컴파일
모델을 훈련하기 전에 필요한 몇 가지 설정이 필요합니다.:

모델을 device로 이동 - 훈련 시 모델을 cpu, gpu로 이동시킵니다.  
옵티마이저(Optimizer)-데이터와 손실 함수를 바탕으로 모델의 업데이트 방법을 결정합니다.  
스케줄러(Scheduler)-학습률을 학습이 진행됨에 따라 적절한 값으로 변경하기 위해 사용합니다.


In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

optimizer = optim.Adadelta(model.parameters(), lr=1.0)
scheduler = StepLR(optimizer, step_size=1, gamma=0.7)

### 모델 훈련
신경망 모델을 훈련하는 단계는 다음과 같습니다:  
.  
model을 학습 상태로 변경합니다. (가중치 업데이트 가능한 상태)  
dataloader에서 뽑은 배치 사이즈만큼의 훈련 데이터를 device로 이동시킵니다.  

훈련 데이터를 모델에 주입합니다-이 예에서는 train_images와 train_labels 배열입니다.  
모델이 이미지와 레이블을 매핑하는 방법을 배웁니다.  

테스트 세트에 대한 모델의 예측을 만듭니다-이 예에서는 test_images 배열입니다.  
이 예측이 test_labels 배열의 레이블과 맞는지 확인합니다.  

Epoch : dataset 반복 횟수  
예를 들어,  
책 몇번 봤어? == 책 몇 epoch 돌렸어?  


손실 함수(Loss function)-훈련 하는 동안 모델의 오차를 측정합니다.  
모델의 학습이 올바른 방향으로 향하도록 이 함수를 최소화해야 합니다.  

In [ ]:
epochs = 10
dry_run = False # 1 배치만 훈련

for epoch in range(1, epochs+1):
    # 학습
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            if dry_run:
                break
    
    # 테스트
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

    scheduler.step()

# 의류 이미지 예측하기

In [ ]:
train_dataset = datasets.FashionMNIST('./fashion_data', train=True, download=True, transform=transform)
test_dataset = datasets.FashionMNIST('./fashion_data', train=False, transform=transform)

**< dataset summary >**  
n_trainset = 60,000 / n_testset = 10,000  
1 image shape = 28 x 28 / 1 label shape = 1

In [ ]:
unique_class = np.unique([label for img, label in train_dataset])
print(unique_class)
len(unique_class)

데이터셋 안에 label의 이름이 없기 때문에, 직접 넣어줍니다.

In [ ]:
label_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.imshow(train_dataset[0][0].reshape(28, 28, 1))
plt.colorbar()
plt.show()

In [ ]:
label_names[train_dataset[0][1]]

In [ ]:
plt.figure(figsize=(20,20))
for i in range(100):
    plt.subplot(10,10,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(train_dataset[i][0].reshape(28, 28, 1), cmap=plt.cm.binary)
    plt.xlabel(label_names[train_dataset[i][1]])
plt.show()

### 모델 구축

In [ ]:
# 기본 모델 = 완전연결계층 = Linear
class MyFirstModel(nn.Module):
    def __init__(self):
        super(MyFirstModel, self).__init__()

        self.model = nn.Sequential(
            nn.Flatten(),
            nn.Linear(784, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 10),
            nn.Softmax()
        )

    def forward(self, x):
        output = self.model(x)
        return output

model = MyFirstModel()

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

optimizer = optim.Adadelta(model.parameters(), lr=1.0)
scheduler = StepLR(optimizer, step_size=1, gamma=0.7)

In [ ]:
epochs = 10
dry_run = False # 1 배치만 훈련

for epoch in range(1, epochs+1):
    # 학습
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            if dry_run:
                break
    
    # 테스트
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

    scheduler.step()